In [31]:
%reset -f
%config InteractiveShell.ast_node_interactivity = 'all'

# Fetching all Air Quality datasets into their dataframes
# Perform immediate concatenation per year
import os
import pandas as pd

# Initialize
dir = 'CAData/'
colidx = [0,2,4,17]     # column indexes to use
innerkeys = ['Date', 'Site ID', 'COUNTY']
dataA2020 = pd.read_csv(dir + 'cf-2020-co.csv', parse_dates=True, usecols=colidx)
dataA2021 = pd.read_csv(dir + 'cf-2021-co.csv', parse_dates=True, usecols=colidx)
dataA2022 = pd.read_csv(dir + 'cf-2022-co.csv', parse_dates=True, usecols=colidx)
with os.scandir(dir) as datasets:
    for dataset in datasets:
        if dataset.is_file() and 'co' not in dataset.name:
            temp = pd.read_csv(dataset, parse_dates=True,
                               usecols=colidx)
            #temp.rename(columns={temp.columns[2]: 'CO Conc (ppm)', temp.columns[5]: 'NO2 Conc (ppb)',temp.columns[7]: 'Ozone Conc (ppm)',temp.columns[9]: 'Mean Pb Conc (ug/m3 SC)',temp.columns[11]: 'Mean PM10 Conc (ug/m3 SC)',temp.columns[13]: 'Mean PM2.5 Conc (ug/m3 LC)',temp.columns[13]: 'SO2 Conc (ppb)'},inplace=True)
            if '2020' in dataset.name:
                # cols_to_use = dataA2020.columns.difference(temp)
                dataA2020 = pd.merge(dataA2020, temp, how='inner', on=innerkeys)
            elif '2021' in dataset.name:
                # cols_to_use = dataA2021.columns.difference(temp)
                dataA2021 = pd.merge(dataA2021, temp, how='inner', on=innerkeys)
            elif '2022' in dataset.name:
                # cols_to_use = dataA2022.columns.difference(temp)
                dataA2022 = pd.merge(dataA2022, temp, how='inner', on=innerkeys)

dataA2020.rename(columns={dataA2020.columns[2]: 'CO Conc (ppm)', dataA2020.columns[4]: 'NO2 Conc (ppb)',dataA2020.columns[5]: 'Ozone Conc (ppm)',dataA2020.columns[6]: 'Pb Conc (ug/m3 SC)',dataA2020.columns[7]: 'PM10 Conc (ug/m3 SC)',dataA2020.columns[8]: 'PM2.5 Conc (ug/m3 LC)',dataA2020.columns[9]: 'SO2 Conc (ppb)'},inplace=True)
#dataA2020 = dataA2020.drop(dataA2020.columns[[3,6,8,10,12,14,16]],axis = 1)

dataA2021.rename(columns={dataA2021.columns[2]: 'CO Conc (ppm)', dataA2021.columns[4]: 'NO2 Conc (ppb)',dataA2021.columns[5]: 'Ozone Conc (ppm)',dataA2021.columns[6]: 'Pb Conc (ug/m3 SC)',dataA2021.columns[7]: 'PM10 Conc (ug/m3 SC)',dataA2021.columns[8]: 'PM2.5 Conc (ug/m3 LC)',dataA2021.columns[9]: 'SO2 Conc (ppb)'},inplace=True)
#dataA2021 = dataA2021.drop(dataA2021.columns[[3,6,8,10,12,14,16]],axis = 1)

dataA2022.rename(columns={dataA2022.columns[2]: 'CO Conc (ppm)', dataA2022.columns[4]: 'NO2 Conc (ppb)',dataA2022.columns[5]: 'Ozone Conc (ppm)',dataA2022.columns[6]: 'Mean Pb Conc (ug/m3 SC)',dataA2022.columns[7]: 'Mean PM10 Conc (ug/m3 SC)',dataA2022.columns[8]: 'Mean PM2.5 Conc (ug/m3 LC)',dataA2022.columns[9]: 'SO2 Conc (ppb)'},inplace=True)
#dataA2022 = dataA2022.drop(dataA2022.columns[[3,6,8,10,12,14,16]],axis = 1)

dataA2020.groupby(by = ['Date', 'Site ID']).mean().groupby(by = ['Date']).mean()
dataA2020
dataA2021
dataA2022

C:\Users\NItro\AppData\Local\Temp\ipykernel_25056\2533960915.py:41: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dataA2020.groupby(by = ['Date', 'Site ID']).mean().groupby(by = ['Date']).mean()


,CO Conc (ppm),NO2 Conc (ppb),Ozone Conc (ppm),Pb Conc (ug/m3 SC),PM10 Conc (ug/m3 SC),PM2.5 Conc (ug/m3 LC),SO2 Conc (ppb)
Date,,,,,,,
01/04/2020,1.170000,31.460000,0.030200,0.008908,38.000000,19.685000,0.740000
01/10/2020,0.750000,33.075000,0.020500,0.006750,29.000000,12.605000,0.800000
01/16/2020,1.000000,36.583333,0.041000,0.010053,57.333333,24.605556,2.700000
01/22/2020,0.400000,29.250000,0.036000,0.003000,27.500000,15.450000,0.500000
01/28/2020,0.625000,34.562500,0.032500,0.004996,38.500000,9.706250,0.875000
01/31/2020,0.800000,28.000000,0.037000,0.006850,23.000000,7.100000,0.400000
02/03/2020,0.150000,7.000000,0.040000,0.004000,40.500000,3.350000,0.200000
02/09/2020,0.380000,16.400000,0.037600,0.002867,21.800000,7.905000,0.320000
02/15/2020,0.700000,34.300000,0.052000,0.006000,49.000000,18.575000,0.900000


,Date,Site ID,CO Conc (ppm),COUNTY,NO2 Conc (ppb),Ozone Conc (ppm),Pb Conc (ug/m3 SC),PM10 Conc (ug/m3 SC),PM2.5 Conc (ug/m3 LC),SO2 Conc (ppb)
0,01/04/2020,60190011,0.8,Fresno,14.9,0.025,0.00065,21,9.4,0.2
1,01/04/2020,60190011,0.8,Fresno,14.9,0.025,0.00065,21,16.0,0.2
2,01/04/2020,60190011,0.8,Fresno,14.9,0.025,0.00065,21,14.6,0.2
3,01/04/2020,60190011,0.8,Fresno,14.9,0.025,0.00065,21,16.0,0.2
4,01/04/2020,60190011,0.8,Fresno,14.9,0.025,0.00065,20,9.4,0.2
...,...,...,...,...,...,...,...,...,...,...
2518,02/09/2020,60850005,0.5,Santa Clara,22.9,0.034,0.00376,25,4.2,1.1
2519,02/21/2020,60850005,1.0,Santa Clara,44.7,0.026,0.01040,39,17.8,1.2
2520,02/21/2020,60850005,1.0,Santa Clara,44.7,0.026,0.01040,39,18.7,1.2
2521,03/04/2020,60850005,0.8,Santa Clara,30.4,0.046,0.00596,34,11.6,1.4


,Date,Site ID,CO Conc (ppm),COUNTY,NO2 Conc (ppb),Ozone Conc (ppm),Pb Conc (ug/m3 SC),PM10 Conc (ug/m3 SC),PM2.5 Conc (ug/m3 LC),SO2 Conc (ppb)
0,04/04/2021,60190011,0.2,Fresno,5.2,0.042,0.00147,23,7.1,0.3
1,04/04/2021,60190011,0.2,Fresno,5.2,0.042,0.00147,23,8.3,0.3
2,04/04/2021,60190011,0.2,Fresno,5.2,0.042,0.00147,23,8.7,0.3
3,04/13/2021,60190011,0.3,Fresno,17.9,0.045,0.00311,43,9.9,0.5
4,04/13/2021,60190011,0.3,Fresno,17.9,0.045,0.00311,43,10.5,0.5
...,...,...,...,...,...,...,...,...,...,...
4443,11/30/2021,60850005,1.3,Santa Clara,36.1,0.015,0.00786,39,22.2,0.9
4444,11/30/2021,60850005,1.3,Santa Clara,36.1,0.015,0.00786,39,22.2,0.9
4445,12/12/2021,60850005,0.4,Santa Clara,5.0,0.034,0.00065,6,3.2,0.2
4446,12/12/2021,60850005,0.4,Santa Clara,5.0,0.034,0.00065,6,2.5,0.2


,Date,Site ID,CO Conc (ppm),COUNTY,NO2 Conc (ppb),Ozone Conc (ppm),Mean Pb Conc (ug/m3 SC),Mean PM10 Conc (ug/m3 SC),Mean PM2.5 Conc (ug/m3 LC),SO2 Conc (ppb)
0,09/15/2022,60190011,0.2,Fresno,4.4,0.049,0.00267,54,11.5,1.0
1,09/15/2022,60190011,0.2,Fresno,4.4,0.049,0.00267,54,14.4,1.0
2,09/26/2022,60190011,0.3,Fresno,15.1,0.068,0.00388,47,9.5,2.3
3,09/26/2022,60190011,0.3,Fresno,15.1,0.068,0.00388,47,9.7,2.3
4,11/01/2022,60190011,1.1,Fresno,27.1,0.035,0.00532,38,13.8,0.5
...,...,...,...,...,...,...,...,...,...,...
9236,12/07/2022,60850005,0.9,Santa Clara,29.3,0.024,0.00460,18,9.8,1.1
9237,12/19/2022,60850005,1.0,Santa Clara,29.8,0.009,0.00508,41,32.9,1.0
9238,12/19/2022,60850005,1.0,Santa Clara,29.8,0.009,0.00508,41,29.5,1.0
9239,12/31/2022,60850005,0.2,Santa Clara,7.0,0.035,0.00065,3,1.9,0.0
